In [1]:
from reid_functions import *

cfg = Config()
TVT, TMO = set_devices((0,))
model = Model(last_conv_stride=cfg.last_conv_stride)
model_w = DataParallel(model)
tri_loss = TripletLoss(margin=cfg.margin)

optimizer = optim.Adam(model.parameters(),
                     lr=cfg.base_lr,
                     weight_decay=cfg.weight_decay)

modules_optims = [model, optimizer]

map_location = (lambda storage, loc: storage)
sd = torch.load(cfg.model_weight_file, map_location=map_location)
load_state_dict(model, sd)

ext = ExtractFeature(model=model_w,TVT=TVT)
root_directory = '/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/ourdata/Database/Dynamic_Database/'
db_name_list, db_feature_list = extract_DB_features(ext,cfg,root_directory)

querrypath = "/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/ourdata/Database/query3/"
# for filename in sorted(os.listdir(querrypath)):
#     filepath = querrypath+filename
#     person = findPerson(filepath, ext, cfg, db_name_list, db_feature_list)
    
#     print(filename, person)

querry_name_list, querry_feature_list = extract_querry_features(ext, cfg, querrypath = "/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/ourdata/Database/query3/")

q_q_dist = compute_dist(querry_feature_list,querry_feature_list, type='cosine')
g_g_dist = compute_dist(db_feature_list,db_feature_list, type='cosine') 
q_g_dist_cos = compute_dist(querry_feature_list,db_feature_list, type='cosine')
q_g_dist_cos = np.ones_like(q_g_dist_cos) - q_g_dist_cos
q_q_dist = np.ones_like(q_q_dist) - q_q_dist
g_g_dist = np.ones_like(g_g_dist) - g_g_dist

reranked_list = re_ranking(q_g_dist_cos, q_q_dist, g_g_dist)

/home/bmsknight/anaconda3/envs/pytorch/lib/python2.7/site-packages/scikit_learn-0.20.0-py2.7-linux-x86_64.egg/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


Keys not found in source state_dict: 
	 base.layer2.0.bn1.num_batches_tracked
	 base.layer4.1.bn3.num_batches_tracked
	 base.layer4.0.bn1.num_batches_tracked
	 base.layer3.5.bn1.num_batches_tracked
	 base.layer3.2.bn2.num_batches_tracked
	 base.layer1.0.bn1.num_batches_tracked
	 base.layer1.1.bn2.num_batches_tracked
	 base.bn1.num_batches_tracked
	 base.layer3.1.bn3.num_batches_tracked
	 base.layer2.0.bn3.num_batches_tracked
	 base.layer2.0.downsample.1.num_batches_tracked
	 base.layer4.0.bn2.num_batches_tracked
	 base.layer2.2.bn3.num_batches_tracked
	 base.layer3.4.bn2.num_batches_tracked
	 base.layer3.4.bn3.num_batches_tracked
	 base.layer1.0.bn3.num_batches_tracked
	 base.layer1.1.bn1.num_batches_tracked
	 base.layer4.0.bn3.num_batches_tracked
	 base.layer1.1.bn3.num_batches_tracked
	 base.layer3.0.bn3.num_batches_tracked
	 base.layer2.3.bn1.num_batches_tracked
	 base.layer3.2.bn3.num_batches_tracked
	 base.layer3.0.bn2.num_batches_tracked
	 base.layer4.2.bn3.num_batches_tracked
	 

In [2]:
querry_name_list.shape

(912, 3)

In [3]:
reranked_list.shape

(912, 33)

In [4]:
db_name_list.shape

(33,)

In [5]:
frames = list(set(querry_name_list[:,2].tolist()))

In [12]:
framelist = reranked_list[querry_name_list[:,2]== frames[0]]
frame_name_list = querry_name_list[querry_name_list[:,2]== frames[0]]

In [15]:
min_index = np.unravel_index(np.argmin(framelist),framelist.shape)
query_id = frame_name_list[min_index[0],0]
person_name = db_name_list[min_index[1]]

In [17]:
print(frames[0])
print(min_index)
print(query_id)
print(person_name)

1140
(1, 22)
person1
harishanth


In [18]:
framelist[min_index[0],:]=1
framelist[:,db_name_list==person_name]=1

In [19]:
framelist

array([[ 0.97683585,  0.86555898,  0.87245017,  0.89732391,  0.9395963 ,
         1.        ,  0.6897158 ,  0.72203082,  0.73053902,  0.71603489,
         0.71394539,  0.88115281,  0.84711319,  0.87596178,  0.8052454 ,
         0.9675023 ,  0.85715288,  0.89485908,  0.87213999,  0.98290092,
         0.87340689,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  0.78232086,  0.8599425 ,  0.84640837,  0.86922765,
         0.81332308,  0.83608347,  0.83244044],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ],
       [ 0.87855506,  0.8298

In [20]:
min_index = np.unravel_index(np.argmin(framelist),framelist.shape)
query_id = frame_name_list[min_index[0],0]
person_name = db_name_list[min_index[1]]

In [21]:
print(frames[0])
print(min_index)
print(query_id)
print(person_name)

1140
(0, 6)
person0
thivakaran


In [22]:
framelist[min_index[0],:]=1
framelist[:,db_name_list==person_name]=1

In [23]:
min_index = np.unravel_index(np.argmin(framelist),framelist.shape)
query_id = frame_name_list[min_index[0],0]
person_name = db_name_list[min_index[1]]

In [24]:
print(frames[0])
print(min_index)
print(query_id)
print(person_name)

1140
(2, 14)
person2
madhushan


In [25]:
framelist[min_index[0],:]=1
framelist[:,db_name_list==person_name]=1

In [26]:
min_index = np.unravel_index(np.argmin(framelist),framelist.shape)
query_id = frame_name_list[min_index[0],0]
person_name = db_name_list[min_index[1]]

In [27]:
print(frames[0])
print(min_index)
print(query_id)
print(person_name)

1140
(4, 4)
person5
piriyanthan


In [28]:
framelist[min_index[0],:]=1
framelist[:,db_name_list==person_name]=1

In [29]:
min_index = np.unravel_index(np.argmin(framelist),framelist.shape)
query_id = frame_name_list[min_index[0],0]
person_name = db_name_list[min_index[1]]

In [30]:
print(frames[0])
print(min_index)
print(query_id)
print(person_name)

1140
(3, 26)
person3
athavan


In [31]:
framelist[min_index[0],:]=1
framelist[:,db_name_list==person_name]=1

In [33]:
framelist

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.]], dtype=float32)

In [58]:
voting_dictionary = {}
voting_name_list = np.asarray(list(set(db_name_list)))
no_of_candidates = voting_name_list.shape[0]
for frame in frames:
    frame_dist_list = reranked_list[querry_name_list[:,2]== frame]
    frame_name_list = querry_name_list[querry_name_list[:,2]== frame]
    no_of_persons = frame_name_list.shape[0]
    for bb in range(no_of_persons):
        min_index = np.unravel_index(np.argmin(frame_dist_list),frame_dist_list.shape)
        query_id = frame_name_list[min_index[0],0]
        person_name = db_name_list[min_index[1]]
        #print(frame,query_id,person_name)
        frame_dist_list[min_index[0],:]=1
        frame_dist_list[:,db_name_list==person_name]=1
        if (query_id not in voting_dictionary.keys()):
            voting_dictionary[query_id] = np.zeros(no_of_candidates)
        voting_dictionary[query_id] = voting_dictionary[query_id] + (voting_name_list==person_name)

In [59]:
voting_dictionary

{'person0': array([ 63.,   9.,   8.,   9.,  98.]),
 'person1': array([ 73.,   5.,  22.,  63.,  17.]),
 'person2': array([ 32.,   7.,  53.,  44.,  26.]),
 'person3': array([ 12.,  18.,  59.,  23.,  20.]),
 'person4': array([  0.,  44.,  25.,  20.,  11.]),
 'person5': array([   1.,  105.,   14.,   22.,    9.])}

In [60]:
voting_name_list

array(['harishanth', 'piriyanthan', 'athavan', 'madhushan', 'thivakaran'], 
      dtype='|S11')

In [48]:
Voting_name_list.shape[0]

5